# Machine Learning

In this file, instructions how to approach the challenge can be found.

We are going to work on different types of Machine Learning problems:

- **Regression Problem**: The goal is to predict delay of flights.
- **(Stretch) Multiclass Classification**: If the plane was delayed, we will predict what type of delay it is (will be).
- **(Stretch) Binary Classification**: The goal is to predict if the flight will be cancelled.

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly 


### *Failed attempts at importing custom modules*

In [73]:
import sys
sys.path.append("C:/Users/silvh/OneDrive/lighthouse/custom python")
import silvhua

In [72]:
import os     
os.environ["PATH"] += os.pathsep + 'C:/Users/silvh/OneDrive/lighthouse/custom python'
import silvhua

### *Custom functions*

In [52]:
# Create function to load CSV files.
def load_csv(filepath,filename,column1_as_index=False):
    """
    Load a csv file as a dataframe using specified file path copied from windows file explorer.
    Back slashes in file path will be converted to forward slashes.
    Arguments:
    - filepath (raw string): Use the format r'<path>'.
    - filename (string).
    - colum1_as_index (bool): If true, take the first column as the index. 
        Useful when importing CSV files from previously exported dataframes.

    Returns: dataframe object.

    Required import: pandas
    """
    filename = f'{filepath}/'.replace('\\','/')+filename
    df = pd.read_csv(filename)
    if column1_as_index==True:
        df.set_index(df.columns[0], inplace=True)
        df.index.name = None
    return df

In [76]:
def compare_id(df1, df1_column, df2, df2_column,print_common=False,print_difference=True):
    """
    Print the number of common values and unique values between two dataframe columns.
    
    """
    df1_values = df1[df1_column].unique()
    df2_values = df2[df2_column].unique()
    common_values = set(df1_values) & set(df2_values)
    if len(df1_values) > len(df2_values):
        different_values = set(df1_values) - set(df2_values)
        print(f'Proper subset = {set(df2_values) < set(df1_values)}')
    else:
        different_values = set(df2_values) - set(df1_values)
        print(f'Proper subset = {set(df1_values) < set(df2_values)}')
    print('Unique values in df1:',len(df1_values))
    print('Unique values in df2:',len(df2_values))
    print('Number of common values between df1 and df2:',len(common_values))
    print('Number of different values between df1 and df2:',len(different_values))
    if print_common == True:
        print('Values in common:',common_values)
    if print_difference == True:
        print('Different values:',different_values)

# function that prints null values
def explore(df,id=0,print_n_unique=True, printValues=False):
    """
    Explore dataframe data and print missing values.
    Parameters:
    - df: Dataframe.
    - id: Column number or name with the primary IDs. Default is zero.
    - print_n_unique (bool): If the number of unique values in the first column doesn't match 
        the number of rows in the df, print the number of unique values in each column to see if 
        there's another column that might serve as a unique id.
    """
    if (id==False) & (id !=0):
        pass
    elif isinstance(id,int):
    # if type(id)==int:
        print(f'Unique IDs: {len(set(df.iloc[:,0]))}. # of rows: {df.shape[0]}. Match: {len(set(df.iloc[:,0]))==df.shape[0]}')
    else:
        print(f'Unique IDs: {len(set(df[id]))}. # of rows: {df.shape[0]}. Match: {len(set(df[id]))==df.shape[0]}')
    
    # if the number of unique values in the first column doesn't match the number of rows in the df,
    # print the number of unique values in each column to see if there's another column that migh
    # serve as a unique id.
    if (print_n_unique==True):
        if len(set(df.iloc[:,0])) !=df.shape[0]: 
            for column in df.columns:
                print(len(df[column].value_counts()),'\t', column)
    
    # count amount of missing values in each column
    total = df.isnull().sum().sort_values(ascending=False) 
    # % of rows with missing data from each column
    percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False) 

    # create a table that lists total and % of missing values starting with the highest
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent']) 

    if (printValues == True):
        # extract the names of columns with missing values
        cols_with_missing = missing_data[missing_data.Percent > 0].index.tolist()
        print(df.dtypes[cols_with_missing])

    print(f'')
    return missing_data

# Function to plot multiple histograms using Plotly. Show different colours based on classification.
def plot_int_hist(df, columns=None, color=None):
    """
    Use Plotly to plot multiple histograms using the specified columns of a dataframe.
    Arguments:
    - df: Dataframe.
    - columns (optional): Columns of dataframe on which to create the histogram. If blank, all numeric data will be plotted.
    - color (optional): Provide name of colum containing binary classification values 0 and 1. 
        Data points classified as 1 will be in red.
    
    Make sure to do the following imports:
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    """
    if columns == None:
        columns = df.dtypes[df.dtypes != 'object'].index.tolist()
    fig = make_subplots(rows=round((len(columns)+.5)/2), cols=2,subplot_titles=columns)
    for i, feature in enumerate(columns):
        if color:
            bins = dict(
                start = min(df[feature]),
                end =  max(df[feature]),
                # size=
            )
            zero = df[df[color]==0]
            one = df[df[color] != 0]
            fig.add_trace(go.Histogram(x=zero[feature],
                marker_color='#330C73',
                opacity=0.5,
                xbins=bins), 
                row=i//2+1, col=i % 2 + 1
                )
            fig.add_trace(go.Histogram(x=one[feature],
                marker_color='red',
                opacity=0.5,
                xbins=bins),
                row=i//2+1, col=i % 2 + 1)
        else:
            fig.add_trace(go.Histogram(x=df[feature]), 
            row=i//2+1, col=i % 2 + 1)
    fig.update_layout(height=300*round((len(columns)+.5)/2), 
        showlegend=False,barmode='overlay')
    fig.show()


    
def correlation(df):
    """
    Plot the correlation matrix.
    Returns the dataframe with the correlation values.
    """

    # Create a mask to exclude the redundant cells that make up half of the graph.
    mask = np.triu(np.ones_like(df.corr(), dtype=bool))

    # Create the heatmap with the mask and with annotation
    sns.heatmap(data=df.corr(numeric_only=True),mask=mask,annot=True)
    return df.corr()

# Function to plot multiple histograms
def plot_hist(df, columns=None):
    """
    Plot multiple histograms using the specified columns of a dataframe.
    Arguments:
    df: Dataframe.
    columns (optional): Columns of dataframe on which to create the histogram. If blank, all numeric data will be plotted.
    
    Make sure to `import seaborn as sns`.
    """
    if columns == None:
        columns = df.dtypes[df.dtypes != 'object'].index.tolist()
    fig, ax = plt.subplots(nrows=round((len(columns)+.5)/2), ncols=2, figsize=(10,18))
    for i, feature in enumerate(columns):
        sns.histplot(data=df,x=feature,ax=ax[i//2, i % 2])
    plt.tight_layout()

### *Load files*

In [55]:

fuel = load_csv(r"C:\Users\silvh\OneDrive\lighthouse\projects\midterm_shared\data\experimentation/",'fuel_features_selected_10-23.csv',column1_as_index=True)
fuel.head()


,month,airline_id,unique_carrier,carrier_group_new,sdomt_gallons,sint_gallons,ts_gallons,tdomt_gallons,tint_gallons,total_gallons,year
0,1,NaN,NaN,1,0.0,0.0,0.0,3000.0,0.0,3000.0,2016
1,1,21352.0,0WQ,1,0.0,0.0,0.0,163052.0,47060.0,210112.0,2016
2,1,21645.0,23Q,1,0.0,0.0,0.0,0.0,0.0,0.0,2016
3,1,21652.0,27Q,1,0.0,0.0,0.0,0.0,0.0,0.0,2016
4,1,20408.0,5V,1,260848.0,0.0,260848.0,284362.0,0.0,284362.0,2016


In [59]:
passengers = load_csv(r'C:\Users\silvh\OneDrive\lighthouse\projects\midterm_shared\data\experimentation','passengers_features_selected_10-23.csv',column1_as_index=True)
passengers

,airline_id,unique_carrier,departures_performed,payload,seats,passengers,freight,mail,distance,air_time,...,dest_airport_id,dest_city_market_id,dest_city_name,dest_country_name,aircraft_group,aircraft_type,aircraft_config,year,month,class
0,21342,VB,34.0,1225224.0,6120.0,6069.0,0.0,0.0,1307.0,0.0,...,11874,31874,"Guadalajara, Mexico",Mexico,6,694,1,2019,6,F
1,21342,VB,5.0,180180.0,900.0,852.0,0.0,0.0,1341.0,0.0,...,11032,31032,"Cancun, Mexico",Mexico,6,694,1,2019,6,L
2,21342,VB,1.0,36036.0,180.0,48.0,0.0,0.0,1330.0,0.0,...,10397,30397,"Atlanta, GA",United States,6,694,1,2019,6,L
3,21342,VB,30.0,1081080.0,5400.0,4140.0,0.0,0.0,2090.0,0.0,...,12478,31703,"New York, NY",United States,6,694,1,2019,6,F
4,21342,VB,30.0,1081080.0,5400.0,4604.0,0.0,0.0,1507.0,0.0,...,12889,32211,"Las Vegas, NV",United States,6,694,1,2019,6,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,19977,UA,1.0,40142.0,179.0,167.0,3225.0,0.0,269.0,41.0,...,12896,32896,"Lubbock, TX",United States,6,888,1,2019,9,L
99996,19977,UA,1.0,45919.0,179.0,75.0,0.0,0.0,163.0,29.0,...,11775,31775,"Sioux Falls, SD",United States,6,888,1,2019,9,F
99997,19977,UA,1.0,40605.0,166.0,77.0,0.0,0.0,416.0,61.0,...,13930,30977,"Chicago, IL",United States,6,614,1,2019,9,F
99998,20336,H6,3.0,10200.0,27.0,22.0,35.0,0.0,126.0,158.0,...,14485,34485,"Red Dog, AK",United States,4,416,3,2019,9,L


### *How to merge data from passengers and fuel tables*

In [81]:
# See how passengers and fuel tables can be merged
compare_id(passengers, 'carrier_group_new',fuel,'carrier_group_new')
print()
compare_id(passengers, 'airline_id',fuel,'airline_id',print_difference=False)

Proper subset = True
Unique values in df1: 8
Unique values in df2: 3
Number of common values between df1 and df2: 3
Number of different values between df1 and df2: 5
Different values: {0, 4, 5, 6, 9}

Proper subset = False
Unique values in df1: 297
Unique values in df2: 63
Number of common values between df1 and df2: 44
Number of different values between df1 and df2: 253


In [79]:
fuel['carrier_group_new'].value_counts()

2    1052
3     999
1     984
Name: carrier_group_new, dtype: int64

In [92]:
fuel.groupby(['year','month']).agg('mean').filter(regex='gallons')

C:\Users\silvh\AppData\Local\Temp\ipykernel_7332\1499054143.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  fuel.groupby(['year','month']).agg('mean').filter(regex='gallons')


sdomt_gallons  sint_gallons    ts_gallons  tdomt_gallons  \
year month                                                             
2015 1       1.476351e+07  8.466170e+06  2.322968e+07   1.499810e+07   
     2       1.348288e+07  7.495644e+06  2.097852e+07   1.374112e+07   
     3       1.617990e+07  8.803838e+06  2.498373e+07   1.652029e+07   
     4       1.607412e+07  8.915791e+06  2.498991e+07   1.633203e+07   
     5       1.654119e+07  9.504175e+06  2.604536e+07   1.680760e+07   
     6       1.707240e+07  9.796820e+06  2.686922e+07   1.733280e+07   
     7       1.793957e+07  1.033557e+07  2.827514e+07   1.817202e+07   
     8       1.740370e+07  1.012692e+07  2.753062e+07   1.768057e+07   
     9       1.558372e+07  9.133133e+06  2.471685e+07   1.590749e+07   
     10      1.669548e+07  9.108164e+06  2.580364e+07   1.699710e+07   
     11      1.585227e+07  8.121753e+06  2.397402e+07   1.613862e+07   
     12      1.752077e+07  8.948919e+06  2.646969e+07   1.788589e+07   
2016 1       1.528046e+07  8.311389e+06  2.359185e+07   1.551980e+07   
     2       1.446864e+07  7.356434e+06  2.182508e+07   1.468971e+07   
     3       1.655956e+07  8.476982e+06  2.503654e+07   1.696333e+07   
     4       1.584829e+07  8.479384e+06  2.432768e+07   1.610379e+07   
     5       1.667492e+07  8.962004e+06  2.563692e+07   1.691256e+07   
     6       1.771874e+07  9.464847e+06  2.718359e+07   1.799665e+07   
     7       1.802816e+07  9.813457e+06  2.784162e+07   1.828654e+07   
     8       1.766195e+07  9.583809e+06  2.724576e+07   1.794169e+07   
     9       1.574188e+07  8.614925e+06  2.435681e+07   1.606422e+07   
     10      1.613522e+07  8.465587e+06  2.460081e+07   1.649804e+07   
     11      1.554464e+07  7.584207e+06  2.312885e+07   1.590107e+07   
     12      1.683441e+07  8.310236e+06  2.514465e+07   1.724723e+07   
2017 1       1.604294e+07  8.380385e+06  2.442333e+07   1.635444e+07   
     2       1.461904e+07  7.345133e+06  2.196417e+07   1.488005e+07   
     3       1.787102e+07  8.908263e+06  2.677928e+07   1.821782e+07   
     4       1.702297e+07  9.050924e+06  2.607389e+07   1.733559e+07   
     5       1.784998e+07  9.689104e+06  2.753908e+07   1.819362e+07   
     6       1.939738e+07  1.047935e+07  2.987673e+07   1.975065e+07   
     7       1.921273e+07  1.062230e+07  2.983503e+07   1.958503e+07   
     8       1.981172e+07  1.075106e+07  3.056277e+07   2.019724e+07   
     9       1.628445e+07  9.153441e+06  2.543789e+07   1.671323e+07   
     10      1.906826e+07  9.908822e+06  2.897708e+07   1.955867e+07   
     11      1.807276e+07  8.799782e+06  2.687254e+07   1.858961e+07   
     12      1.989829e+07  9.812215e+06  2.971051e+07   2.048928e+07   
2018 1       1.858604e+07  9.562607e+06  2.814864e+07   1.943596e+07   
     2       1.736592e+07  8.498592e+06  2.586451e+07   1.813659e+07   
     3       1.980512e+07  9.854823e+06  2.965995e+07   2.027998e+07   
     4       1.969482e+07  9.994447e+06  2.968927e+07   2.015247e+07   
     5       2.023683e+07  1.062547e+07  3.086231e+07   2.069258e+07   
     6       2.077278e+07  1.097070e+07  3.174348e+07   2.122587e+07   
     7       2.156843e+07  1.151516e+07  3.308359e+07   2.201548e+07   
     8       2.142221e+07  1.119067e+07  3.261288e+07   2.189511e+07   
     9       1.896372e+07  1.006732e+07  2.903104e+07   1.947985e+07   
     10      2.007686e+07  1.002813e+07  3.010500e+07   2.066607e+07   
     11      1.903211e+07  8.834545e+06  2.786666e+07   1.967617e+07   
     12      1.973893e+07  9.461856e+06  2.920079e+07   2.041265e+07   
2019 1       1.858145e+07  9.330355e+06  2.791180e+07   1.911017e+07   
     2       1.749402e+07  8.318555e+06  2.581258e+07   1.786542e+07   
     3       2.023282e+07  9.693282e+06  2.992610e+07   2.084932e+07   
     4       1.945740e+07  9.681625e+06  2.913902e+07   2.003593e+07   
     5       2.053160e+07  1.051912e+07  3.105072e+07   2.110082e+07   
     6       2.078645e+07  1.088544e+07  3.167189e+0

## Main Task: Regression Problem

The target variable is **ARR_DELAY**. We need to be careful which columns to use and which don't. For example, DEP_DELAY is going to be the perfect predictor, but we can't use it because in real-life scenario, we want to predict the delay before the flight takes of --> We can use average delay from earlier days but not the one from the actual flight we predict.  

For example, variables **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY** shouldn't be used directly as predictors as well. However, we can create various transformations from earlier values.

We will be evaluating your models by predicting the ARR_DELAY for all flights **1 week in advance**.

In [58]:
from datetime import datetime
# Convert date to cycling number
# current_date = "2022-01-01 00:00:00"
# cdate = datetime.strptime(current_date, '%Y-%m-%d %H:%M:%S')
# day_sin = np.sin(2 * np.pi * cdate.timetuple().tm_yday/365.0)
# day_cos = np.cos(2 * np.pi * cdate.timetuple().tm_yday/365.0)
# day_cos

### Feature Engineering

Feature engineering will play a crucial role in this problems. We have only very little attributes so we need to create some features that will have some predictive power.

- weather: we can use some weather API to look for the weather in time of the scheduled departure and scheduled arrival.
- statistics (avg, mean, median, std, min, max...): we can take a look at previous delays and compute descriptive statistics
- airports encoding: we need to think about what to do with the airports and other categorical variables
- time of the day: the delay probably depends on the airport traffic which varies during the day.
- airport traffic
- unsupervised learning as feature engineering?
- **what are the additional options?**: Think about what we could do more to improve the model.

### Feature Selection / Dimensionality Reduction

We need to apply different selection techniques to find out which one will be the best for our problems.

- Original Features vs. PCA conponents?

### Modeling

Use different ML techniques to predict each problem.

- linear / logistic / multinomial logistic regression
- Naive Bayes
- Random Forest
- SVM
- XGBoost
- The ensemble of your own choice

### Evaluation

You have data from 2018 and 2019 to develop models. Use different evaluation metrics for each problem and compare the performance of different models.

You are required to predict delays on **out of sample** data from **first 7 days (1st-7th) of January 2020** and to share the file with LighthouseLabs. Sample submission can be found in the file **_sample_submission.csv_**

======================================================================
## Stretch Tasks

### Multiclass Classification

The target variables are **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY**. We need to do additional transformations because these variables are not binary but continuos. For each flight that was delayed, we need to have one of these variables as 1 and others 0.

It can happen that we have two types of delays with more than 0 minutes. In this case, take the bigger one as 1 and others as 0.

### Binary Classification

The target variable is **CANCELLED**. The main problem here is going to be huge class imbalance. We have only very little cancelled flights with comparison to all flights. It is important to do the right sampling before training and to choose correct evaluation metrics.